In [5]:
import pandas as pd

In [6]:
data=pd.read_csv('processed_data_all.csv')
data.head(10)

,content
0,Kumaha barudak? Well!Yuk ramaikan Pesta Anak N...
1,Kustini Sri Purnomo memiliki strategi jitu unt...
2,"Peraih Emas Olimpiade Tokyo, Greysia Polii bic..."
3,Kapolri Jenderal Lisyto Sigit Prabowo bicara a...
4,"Menteri BUMN, Erick Thohir bicara bangsa Indon..."
5,Iring-iringan mobil SUV pasukan pengawal Presi...
6,Seorang remaja Palestina umur 14 tahun ditemba...
7,"Ketua Umum PDIP, Megawati Soekarnoputri mengin..."
8,Sekjen PDIP Hasto Kristiyanto berbicara terkai...
9,Presiden Joko Widodo memimpin upacara peringat...


In [7]:
len(data)

27390

In [8]:
from concurrent.futures import ThreadPoolExecutor
from transformers import pipeline

# Load model analisis sentimen bahasa Indonesia dengan tiga kelas sentimen
sentiment_analysis = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")


In [9]:
# Fungsi untuk melabeli sentimen pada setiap baris data
def label_sentiment(text):
    # Analisis sentimen menggunakan model Transformer
    result = sentiment_analysis(text)
    # Ambil label sentimen teratas
    top_sentiment = result[0]
    label = top_sentiment['label']
    score = top_sentiment['score']
    print(label)
    return label, score

# Fungsi untuk melabeli sentimen menggunakan multi-threading
def label_sentiment_multithread(text_list):
    with ThreadPoolExecutor() as executor:
        results = executor.map(label_sentiment, text_list)
    return list(results)

# Bagi data menjadi batch agar dapat diproses menggunakan multi-threading
batch_size = 1000
num_batches = len(data) // batch_size + 1
batches = [data[i*batch_size:(i+1)*batch_size] for i in range(num_batches)]

# Melabeli sentimen untuk setiap batch menggunakan multi-threading
labeled_data = []
for batch in batches:
    texts = batch['content'].tolist()
    labels = label_sentiment_multithread(texts)
    labeled_data.extend(labels)

# Memasukkan hasil label sentimen ke dalam DataFrame
data['sentiment_label'], data['sentiment_score'] = zip(*labeled_data)

Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Positive
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Negative
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Negative
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutr

In [ ]:
# Simpan data yang telah dilabeli ke dalam file CSV
data.to_csv('label_all.csv', index=False)